In [1]:
import numpy as np
import pandas as pd
import gc
import os
import matplotlib.pyplot as plt
import polars as pl
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import Parallel, delayed
from more_itertools import chunked
from functools import reduce
from typing import List
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import joblib
import os
import torch.nn as nn

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

True
1
NVIDIA GeForce RTX 3060


In [3]:
df_full = pd.read_parquet('./data/product_train_val_NN_TORCH.parquet', engine='fastparquet')

In [4]:
# Separar conjuntos
df_train = df_full[df_full['PERIODO'] <= 201909].copy()
df_val = df_full[(df_full['PERIODO'] == 201910)].copy()
df_pred = df_full[df_full['PERIODO'] == 201912].copy()
del df_full
gc.collect()

20

In [5]:
import joblib

target_col = 'CLASE_LOG1P_Z'
cat_cols = ['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'MES_PROBLEMATICO', 'PRODUCT_RANK_BIN', 
            'IS_FEBRERO', 'ESTOY_PREDICIENDO_FEBRERO', 'CAIDA_ABRUPTA']
label_encoders = {}

# Cargar los encoders entrenados
for col in cat_cols:
    le = joblib.load(f'encoders/{col}_encoder.pkl')
    label_encoders[col] = le

    # Transformar los datasets (train, val, pred) usando ese encoder
    for df in [df_train, df_val, df_pred]:
        df[col] = df[col].map(lambda x: le.transform([x])[0] if x in le.classes_ else 0)

# Definir embedding_sizes (lo mejor es usar el encoder para ver la cantidad de clases)
embedding_sizes = [
    (len(label_encoders[col].classes_) + 1, min(50, (len(label_encoders[col].classes_) + 1) // 2))
    for col in cat_cols
]

# Excluir columnas que no deben ir al modelo
excluir = ['PERIODO', 'CUSTOMER_ID', 'PRODUCT_ID', 'CLASE_LOG1P_Z', 'ORDINAL']
feature_cols = [col for col in df.columns if col not in excluir and col not in cat_cols]

In [6]:
# No los incluyas en ninguna de estas dos listas
assert 'CUSTOMER_ID' not in feature_cols
assert 'CUSTOMER_ID' not in cat_cols
assert 'PRODUCT_ID' not in feature_cols
assert 'PRODUCT_ID' not in cat_cols
assert 'PERIODO' not in feature_cols
assert 'PERIODO' not in cat_cols
assert 'CLASE_LOG1P_Z' not in feature_cols
assert 'CLASE_LOG1P_Z' not in cat_cols
assert 'ORDINAL' not in feature_cols
assert 'ORDINAL' not in cat_cols


In [7]:
assert all(col in df_train.columns for col in cat_cols), "Faltan columnas categóricas"
assert all(col in df_train.columns for col in feature_cols), "Faltan columnas numéricas"
assert target_col in df_train.columns, "Falta la variable objetivo"

In [8]:
from torch.utils.data import Dataset
import torch

class TabularDataset(Dataset):
    def __init__(self, df, cat_cols, num_cols, target_col=None):
        self.cat_data = torch.tensor(df[cat_cols].values, dtype=torch.long)
        self.num_data = torch.tensor(df[num_cols].values, dtype=torch.float32)
        self.has_target = target_col is not None
        if self.has_target:
            self.y = torch.tensor(df[target_col].values, dtype=torch.float32).unsqueeze(1)
        else:
            self.y = None

    def __len__(self):
        return len(self.cat_data)

    def __getitem__(self, idx):
        if self.has_target:
            return self.cat_data[idx], self.num_data[idx], self.y[idx]
        else:
            return self.cat_data[idx], self.num_data[idx]

In [9]:
batch_size = 4096

train_dataset = TabularDataset(df_train, cat_cols, feature_cols, target_col)
val_dataset = TabularDataset(df_val, cat_cols, feature_cols, target_col)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [10]:
print(cat_cols)
print(feature_cols)
print(target_col)

['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'MES_PROBLEMATICO', 'PRODUCT_RANK_BIN', 'IS_FEBRERO', 'ESTOY_PREDICIENDO_FEBRERO', 'CAIDA_ABRUPTA']
['MES_SIN', 'MES_COS', 'ORDINAL_Z', 'ANIO_Z', 'MES_Z', 'TRIMESTRE_Z', 'PRODUCT_ID_Z', 'TN_Z', 'PROM_ULT_3_FEBREROS_Z', 'DIF_TN_VS_FEBREROS_ULT_3_Z', 'TN_MAX_HISTORICO_Z', 'TN_DIST_A_MAX_HIST_Z', 'TN_RATIO_VS_MAX_HIST_Z', 'TN_MEAN_03_Z', 'PENDIENTE_TENDENCIA_3_Z', 'TN_EWMA_03_Z', 'TN_MEDIAN_03_Z', 'TN_MIN_03_Z', 'TN_MAX_03_Z', 'TN_STD_03_Z', 'TN_SKEW_03_Z', 'TN_KURT_03_Z', 'TN_GROWTH_03_Z', 'TN_IQR_03_Z', 'TN_SUM_03_Z', 'TN_COUNT_POS_03_Z', 'TN_PCT_ZERO_03_Z', 'TN_LAST_03_Z', 'TN_LAST_DIFF_03_Z', 'TN_COEF_VAR_3_Z', 'TN_MAXMIN_RATIO_3_Z', 'TN_RANGO_3_Z', 'TN_RANGO_REL_3_Z', 'TN_LAST_VS_MEDIAN_3_Z', 'TN_CHANGE_PREV_WINDOW_3_Z', 'TN_ZEROS_END_3_Z', 'TN_LAST_PCT_SUM_3_Z', 'TN_PCT90_3_Z', 'TN_PCT10_3_Z', 'TN_PCT_WIDTH_3_Z', 'TN_MINUS_MEAN_03_Z', 'TN_MINUS_MEDIAN_03_Z', 'TN_MINUS_EWMA_03_Z', 'TN_OVER_MEAN_03_Z', 'TN_OVER_MEDIAN_03_Z', 'T

In [11]:
import torch.nn.functional as F

class TabularNNImproved(nn.Module):
    def __init__(self, embedding_sizes, num_numerical, hidden_sizes=[512, 512, 256, 128], dropout=0.1):
        super().__init__()
        
        # Embedding layers
        self.embeddings = nn.ModuleList([
            nn.Embedding(ni, nf) for ni, nf in embedding_sizes
        ])
        embedding_dim = sum([nf for _, nf in embedding_sizes])
        self.embedding_dropout = nn.Dropout(dropout)

        # Total input size after embedding + numerical
        input_size = embedding_dim + num_numerical

        # Hidden layers
        layers = []
        for h in hidden_sizes:
            layers.append(nn.Linear(input_size, h))
            layers.append(nn.BatchNorm1d(h))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
            input_size = h

        # Output layer
        layers.append(nn.Linear(input_size, 1))
        self.model = nn.Sequential(*layers)

    def forward(self, x_cat, x_num):
        x = [emb(x_cat[:, i]) for i, emb in enumerate(self.embeddings)]
        x = torch.cat(x, dim=1)
        x = self.embedding_dropout(x)
        x = torch.cat([x, x_num], dim=1)
        return self.model(x)


In [12]:
import torch

# Detectar si hay GPU disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Crear el modelo
model = TabularNNImproved(
    embedding_sizes=embedding_sizes,
    num_numerical=len(feature_cols),
    hidden_sizes=[4096,2048,1024,512, 512, 256, 128],
    dropout=0.3
).to(device)
model.to(device)
print(model)


TabularNNImproved(
  (embeddings): ModuleList(
    (0): Embedding(5, 2)
    (1): Embedding(16, 8)
    (2): Embedding(85, 42)
    (3): Embedding(36, 18)
    (4): Embedding(68, 34)
    (5): Embedding(3, 1)
    (6): Embedding(11, 5)
    (7-9): 3 x Embedding(3, 1)
  )
  (embedding_dropout): Dropout(p=0.3, inplace=False)
  (model): Sequential(
    (0): Linear(in_features=324, out_features=4096, bias=True)
    (1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=4096, out_features=2048, bias=True)
    (5): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU(approximate='none')
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=2048, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): GELU(approximate='none')
    (11): Drop

In [13]:
import torch
import torch.nn as nn

class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, y_pred, y_true):
        return torch.sqrt(torch.mean((y_pred - y_true) ** 2))


In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import torch
import numpy as np

def train_model(
    model, train_loader, val_loader=None, n_epochs=20, lr=1e-3, patience=3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = RMSELoss()
    best_val_loss = float('inf')
    best_model_state = None
    epochs_without_improvement = 0

    best_epoch = 0  # <--- Nuevo: para trackear la mejor epoch
    best_y_true = None
    best_y_pred = None

    for epoch in range(n_epochs):
        # Entrenamiento
        model.train()
        train_loss = 0.0
        for cats, conts, y in train_loader:
            cats, conts, y = cats.to(device), conts.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(cats, conts)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * y.size(0)
        train_loss /= len(train_loader.dataset)

        # Validación (solo si hay val_loader)
        if val_loader is not None:
            model.eval()
            val_loss = 0.0
            y_true_list = []
            y_pred_list = []
            with torch.no_grad():
                for cats, conts, y in val_loader:
                    cats, conts, y = cats.to(device), conts.to(device), y.to(device)
                    y_pred = model(cats, conts)
                    loss = criterion(y_pred, y)
                    val_loss += loss.item() * y.size(0)
                    y_true_list.append(y.cpu().numpy())
                    y_pred_list.append(y_pred.cpu().numpy())

            val_loss /= len(val_loader.dataset)
            y_true = np.concatenate(y_true_list)
            y_pred = np.concatenate(y_pred_list)

            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            r2 = r2_score(y_true, y_pred)

            print(
                f"Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | MAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f}"
            )

            # Early stopping y trackeo de la mejor epoch
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict()
                best_epoch = epoch + 1  # <--- Guardamos la mejor epoch (1-based)
                best_y_true = y_true
                best_y_pred = y_pred
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1
                if patience is not None and epochs_without_improvement >= patience:
                    print("🔴 Early stopping triggered")
                    break
        else:
            # Si no hay val_loader, solo mostrar train_loss
            print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss:.4f}")

    # Restaurar el mejor modelo solo si hubo validación
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    # Retornar también la mejor epoch y sus preds
    if val_loader is not None:
        return best_y_true, best_y_pred, best_epoch
    else:
        return None, None, None


# Búsqueda de hiperparámetros (Grid Search)
Probamos distintas combinaciones de hiperparámetros y seleccionamos la que da mejor MAE en validación.


In [15]:
import torch
import gc
import pandas as pd
import numpy as np
import random
from sklearn.metrics import mean_absolute_error
from itertools import product

# ---- FUNCIÓN DE STRATIFIED SAMPLING ----
def stratified_sampling_grid(param_grid, stratify_by, N_TOTAL, seed=42):
    keys = list(param_grid.keys())
    all_combos = [
        dict(zip(keys, vals))
        for vals in product(*(param_grid[k] for k in keys))
    ]
    df = pd.DataFrame(all_combos)

    # --- Nueva columna hashable para deduplicar y comparar ---
    for col in stratify_by:
        if df[col].apply(lambda x: isinstance(x, list)).any():
            df[col + '_str'] = df[col].apply(lambda x: str(x))
    # Usar '_str' si existe, sino el nombre original
    stratify_keys = [col + '_str' if (col + '_str') in df.columns else col for col in stratify_by]

    random.seed(seed)
    uniq_strata = df.drop_duplicates(subset=stratify_keys)
    uniq_strata = uniq_strata.sample(frac=1, random_state=seed)  # shuffle

    sampled_rows = []
    for _, row in uniq_strata.iterrows():
        cond = pd.Series(True, index=df.index)
        for col in stratify_by:
            key = col + '_str' if (col + '_str') in df.columns else col
            val = str(row[col]) if key.endswith('_str') else row[col]
            cond &= (df[key] == val)
        group = df[cond]
        sampled_rows.append(group.sample(1, random_state=random.randint(0, 99999)).iloc[0])

    if len(sampled_rows) < N_TOTAL:
        ids_usados = set()
        for row in sampled_rows:
            row_tuple = tuple(
                str(row[k]) if (k + '_str') in df.columns else row[k] for k in keys
            )
            ids_usados.add(row_tuple)
        resto = df[
            ~df.apply(lambda r: tuple(
                str(r[k]) if (k + '_str') in df.columns else r[k] for k in keys
            ) in ids_usados, axis=1)
        ]
        faltan = N_TOTAL - len(sampled_rows)
        if len(resto) > 0:
            sampled_rows += list(resto.sample(n=min(faltan, len(resto)), random_state=seed+1).to_dict('records'))

    sampled_rows = sampled_rows[:N_TOTAL]
    # --- Devolver solo las columnas originales ---
    # ¡Asegurarse de que todos sean dicts!
    sampled_rows = [dict(r) for r in sampled_rows]
    result = pd.DataFrame(sampled_rows)[keys].reset_index(drop=True)
    return result

# ---- DEFINICIÓN DEL GRID Y PARÁMETROS ----
param_grid = {
    'lr': [1e-3, 5e-4, 2e-4, 1e-4],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'hidden_sizes': [
        [512, 256], [1024, 512, 256], [2048, 1024, 512, 256], [512, 256, 128],
        [1024, 512, 256, 128], [4096, 2048, 1024, 512, 256], [256, 128],
        [512, 512, 256], [512, 512, 512], [256, 256], [1024, 256, 1024]
    ],
    'seed': [101, 307, 1009, 2029, 5003, 7777, 9871]
}
N_TOTAL = 300  # Ajustá según tus recursos
stratify_by = ['lr', 'dropout', 'hidden_sizes']

# ---- GENERAR COMBINACIONES A ENTRENAR ----
combos_to_run = stratified_sampling_grid(param_grid, stratify_by=stratify_by, N_TOTAL=N_TOTAL)

# ---- LOOP DE ENTRENAMIENTO ----
results = []
best_rmse = float('inf')

for i, row in combos_to_run.iterrows():
    lr = row['lr']
    dropout = row['dropout']
    hidden_sizes = row['hidden_sizes']
    seed = int(row['seed'])

    print(f"\n🔧 [{i+1}/{len(combos_to_run)}] Entrenando con: lr={lr}, dropout={dropout}, hidden_sizes={hidden_sizes}, seed={seed}")

    # Setear semillas
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Crear modelo y mover a dispositivo
    model = TabularNNImproved(
        embedding_sizes=embedding_sizes,
        num_numerical=len(feature_cols),
        hidden_sizes=hidden_sizes,
        dropout=dropout
    ).to(device)

    # Entrenamiento (ajustá la función train_model según tu código)
    y_true_gs, y_pred_gs, best_epoch = train_model(
        model, train_loader, val_loader,
        n_epochs=20, lr=lr, patience=4)

    rmse = np.sqrt(np.mean((np.array(y_true_gs) - np.array(y_pred_gs)) ** 2))
    mae = mean_absolute_error(y_true_gs, y_pred_gs)

    results.append({
        'lr': lr,
        'dropout': dropout,
        'hidden_sizes': hidden_sizes,
        'seed': seed,
        'rmse': rmse,
        'mae': mae,
        'best_epoch': best_epoch
    })

    print(f"✅ RMSE = {rmse:.4f} | MAE = {mae:.4f}")

    if rmse < best_rmse:
        best_rmse = rmse
        torch.save(model.state_dict(), f"best_model_rmse{rmse:.4f}_lr{lr}_do{dropout}_seed{seed}.pth")
        print("💾 Modelo guardado (mejor hasta ahora)")

    del model
    torch.cuda.empty_cache()
    gc.collect()

# ---- GUARDAR Y MOSTRAR RESULTADOS ----
results_df = pd.DataFrame(results).sort_values(by='rmse')
print("\n📊 Mejores combinaciones:")
print(results_df.head(50))
results_df.to_csv("gridsearch_results.csv", index=False)



🔧 [1/300] Entrenando con: lr=0.0002, dropout=0.3, hidden_sizes=[512, 256], seed=5003
Epoch 1/20 | Train Loss: 0.9788 | Val Loss: 0.8006 | MAE: 0.6728 | RMSE: 0.8006 | R²: 0.3138
Epoch 2/20 | Train Loss: 0.7188 | Val Loss: 0.6375 | MAE: 0.5333 | RMSE: 0.6375 | R²: 0.5649
Epoch 3/20 | Train Loss: 0.5721 | Val Loss: 0.5773 | MAE: 0.4549 | RMSE: 0.5773 | R²: 0.6432
Epoch 4/20 | Train Loss: 0.5231 | Val Loss: 0.4381 | MAE: 0.3349 | RMSE: 0.4381 | R²: 0.7945
Epoch 5/20 | Train Loss: 0.4730 | Val Loss: 0.3365 | MAE: 0.2558 | RMSE: 0.3365 | R²: 0.8788
Epoch 6/20 | Train Loss: 0.4506 | Val Loss: 0.3366 | MAE: 0.2480 | RMSE: 0.3366 | R²: 0.8787
Epoch 7/20 | Train Loss: 0.4361 | Val Loss: 0.3208 | MAE: 0.2383 | RMSE: 0.3208 | R²: 0.8898
Epoch 8/20 | Train Loss: 0.4175 | Val Loss: 0.2978 | MAE: 0.2168 | RMSE: 0.2978 | R²: 0.9051
Epoch 9/20 | Train Loss: 0.4062 | Val Loss: 0.2890 | MAE: 0.2056 | RMSE: 0.2890 | R²: 0.9106
Epoch 10/20 | Train Loss: 0.3920 | Val Loss: 0.2950 | MAE: 0.2065 | RMSE: 0.2

In [16]:
import torch
import math
from torch.utils.data import ConcatDataset, DataLoader

# ------------------------
# Prepara el dataset final
# ------------------------
# train_dataset y val_dataset ya deberían estar definidos
train_val_dataset = ConcatDataset([train_dataset, val_dataset])

batch_size = 1024
train_loader_full = DataLoader(train_val_dataset, batch_size=batch_size, shuffle=True)


In [17]:
import pandas as pd
import ast
import json

# --- CONFIGURACIÓN ---
csv_path = "gridsearch_results.csv"
ensemble_size = 50
top_n = 100  # Top-N por métrica combinada a considerar para maximizar diversidad

# --- CARGA Y PREPROCESAMIENTO ---
df = pd.read_csv(csv_path)

# Normalización z-score de métricas
df['mae_z'] = (df['mae'] - df['mae'].mean()) / df['mae'].std()
df['rmse_z'] = (df['rmse'] - df['rmse'].mean()) / df['rmse'].std()
df['score'] = df['mae_z'] + df['rmse_z']

# Selección top-N mejores por score combinado
df_top = df.sort_values('score').head(top_n).copy()

# Convertir string a lista en 'hidden_sizes' si corresponde
def safe_eval(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except Exception:
            return val  # Si falla, lo deja como está
    return val

df_top['hidden_sizes'] = df_top['hidden_sizes'].apply(safe_eval)
# Crear columna auxiliar como string para deduplicar
df_top['hidden_sizes_str'] = df_top['hidden_sizes'].apply(str)

# Elegir modelos diversos: por combinación única de (lr, dropout, hidden_sizes_str)
diverse = df_top.drop_duplicates(subset=['lr', 'dropout', 'hidden_sizes_str'])

# Si hay menos de ensemble_size, completar con otros buenos modelos (distinta seed)
if len(diverse) < ensemble_size:
    faltan = ensemble_size - len(diverse)
    restantes = df_top[~df_top.index.isin(diverse.index)].sort_values('score')
    # Elegir por combinación diferente de seed
    ya = set(tuple(x) for x in diverse[['lr', 'dropout', 'hidden_sizes_str']].values)
    extras = []
    for _, row in restantes.iterrows():
        key = (row['lr'], row['dropout'], row['hidden_sizes_str'])
        if key not in ya:
            extras.append(row)
            ya.add(key)
        if len(extras) >= faltan:
            break
    diverse = pd.concat([diverse, pd.DataFrame(extras)], ignore_index=True)
else:
    diverse = diverse.head(ensemble_size)

# --- CREACIÓN DE CONFIGS ---
configs = []
for i, row in diverse.reset_index(drop=True).iterrows():
    name = f"mlp_lr_{row['lr']}_drop_{row['dropout']}_hs_{'-'.join(map(str, row['hidden_sizes']))}_seed_{int(row['seed'])}"
    name_short = f"modelo_{i+1}"
    configs.append({
        "lr": float(row['lr']),
        "dropout": float(row['dropout']),
        "hidden_sizes": list(row['hidden_sizes']),
        "seed": int(row['seed']),
        "n_epochs": int(row['best_epoch']),
        "name": name_short,
        "name_desc": name
    })

# --- IMPRESIÓN DE RESUMEN ---
print(f"\nModelos seleccionados para ensemble (total={len(configs)}):")
for cfg in configs:
    print(cfg)

print("\nResumen por (lr, dropout):")
print(diverse.groupby(['lr', 'dropout']).size().unstack(fill_value=0))

# (Opcional) Guardar configs a archivo para uso posterior
with open("ensemble_configs.json", "w") as f:
    json.dump(configs, f, indent=2)



Modelos seleccionados para ensemble (total=50):
{'lr': 0.0005, 'dropout': 0.1, 'hidden_sizes': [1024, 512, 256, 128], 'seed': 7777, 'n_epochs': 7, 'name': 'modelo_1', 'name_desc': 'mlp_lr_0.0005_drop_0.1_hs_1024-512-256-128_seed_7777'}
{'lr': 0.0002, 'dropout': 0.1, 'hidden_sizes': [4096, 2048, 1024, 512, 256], 'seed': 5003, 'n_epochs': 7, 'name': 'modelo_2', 'name_desc': 'mlp_lr_0.0002_drop_0.1_hs_4096-2048-1024-512-256_seed_5003'}
{'lr': 0.0002, 'dropout': 0.1, 'hidden_sizes': [2048, 1024, 512, 256], 'seed': 307, 'n_epochs': 7, 'name': 'modelo_3', 'name_desc': 'mlp_lr_0.0002_drop_0.1_hs_2048-1024-512-256_seed_307'}
{'lr': 0.0001, 'dropout': 0.1, 'hidden_sizes': [4096, 2048, 1024, 512, 256], 'seed': 2029, 'n_epochs': 7, 'name': 'modelo_4', 'name_desc': 'mlp_lr_0.0001_drop_0.1_hs_4096-2048-1024-512-256_seed_2029'}
{'lr': 0.001, 'dropout': 0.3, 'hidden_sizes': [512, 256, 128], 'seed': 2029, 'n_epochs': 11, 'name': 'modelo_5', 'name_desc': 'mlp_lr_0.001_drop_0.3_hs_512-256-128_seed_2029

In [18]:
# ------------------------
# Entrenamiento y guardado
# ------------------------

def set_seed(seed):
    import random
    import numpy as np
    import torch
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

for cfg in configs:
    print(f"\n🔧 Entrenando {cfg['name']} - lr={cfg['lr']} dropout={cfg['dropout']} hidden={cfg['hidden_sizes']} seed={cfg['seed']} epochs={cfg['n_epochs']}")
    
    set_seed(cfg['seed'])

    model = TabularNNImproved(
        embedding_sizes=embedding_sizes,
        num_numerical=len(feature_cols),
        hidden_sizes=cfg['hidden_sizes'],
        dropout=cfg['dropout']
    ).to(device)

    # Entrena sobre TODO el set (sin early stopping)
    y_true, y_pred, best_epoch = train_model(
        model,
        train_loader_full,
        val_loader=None,
        n_epochs=cfg['n_epochs'],
        lr=cfg['lr'],
        patience=None  # No early stopping en entrenamiento final
        # Quitar alpha y penalty_indices/coefficients si ya no los usás
    )

    torch.save(model.state_dict(), f"{cfg['name']}_final.pt")
    print(f"💾 Modelo guardado: {cfg['name']}_final.pt")

print("\n✅ ¡Entrenamiento y guardado de los 50 mejores modelos finalizado!")




🔧 Entrenando modelo_1 - lr=0.0005 dropout=0.1 hidden=[1024, 512, 256, 128] seed=7777 epochs=7
Epoch 1/7 | Train Loss: 0.4411
Epoch 2/7 | Train Loss: 0.3021
Epoch 3/7 | Train Loss: 0.2766
Epoch 4/7 | Train Loss: 0.2624
Epoch 5/7 | Train Loss: 0.2543
Epoch 6/7 | Train Loss: 0.2501
Epoch 7/7 | Train Loss: 0.2431
💾 Modelo guardado: modelo_1_final.pt

🔧 Entrenando modelo_2 - lr=0.0002 dropout=0.1 hidden=[4096, 2048, 1024, 512, 256] seed=5003 epochs=7
Epoch 1/7 | Train Loss: 0.4394
Epoch 2/7 | Train Loss: 0.3036
Epoch 3/7 | Train Loss: 0.2873
Epoch 4/7 | Train Loss: 0.2744
Epoch 5/7 | Train Loss: 0.2608
Epoch 6/7 | Train Loss: 0.2496
Epoch 7/7 | Train Loss: 0.2455
💾 Modelo guardado: modelo_2_final.pt

🔧 Entrenando modelo_3 - lr=0.0002 dropout=0.1 hidden=[2048, 1024, 512, 256] seed=307 epochs=7
Epoch 1/7 | Train Loss: 0.4778
Epoch 2/7 | Train Loss: 0.3125
Epoch 3/7 | Train Loss: 0.2883
Epoch 4/7 | Train Loss: 0.2755
Epoch 5/7 | Train Loss: 0.2611
Epoch 6/7 | Train Loss: 0.2593
Epoch 7/7 | Tr

In [19]:
from torch.utils.data import DataLoader, TensorDataset

# --- Predicción MLP (PyTorch) (50 modelos) ---

# Asegurarse de que las categóricas fueron correctamente encodeadas ANTES de esto.

for col in cat_cols:
    df_pred[col] = df_pred[col].astype(np.int64)
for col in feature_cols:
    df_pred[col] = df_pred[col].astype(np.float32)

X_cats = torch.LongTensor(df_pred[cat_cols].values)
X_conts = torch.FloatTensor(df_pred[feature_cols].values)
ds_pred = TensorDataset(X_cats, X_conts)
pred_loader = DataLoader(ds_pred, batch_size=8192, shuffle=False)

def predict_model(model, loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for X_cats, X_conts in loader:
            X_cats = X_cats.to(device)
            X_conts = X_conts.to(device)
            output = model(X_cats, X_conts)
            preds.append(output.cpu().numpy().reshape(-1))
    return np.concatenate(preds)

# Inicializar DataFrame solo con PRODUCT_ID
df_preds_final = pd.DataFrame({'PRODUCT_ID': df_pred['PRODUCT_ID'].values})

for i, cfg in enumerate(configs):
    print(f"Prediciendo MLP {i+1}/50...")
    model = TabularNNImproved(
        embedding_sizes=embedding_sizes,
        num_numerical=len(feature_cols),
        hidden_sizes=cfg['hidden_sizes'],
        dropout=cfg['dropout']
    ).to(device)
    model.load_state_dict(torch.load(f"{cfg['name']}_final.pt", map_location=device))
    preds = predict_model(model, pred_loader, device)
    # Ir agregando cada predicción como nueva columna
    df_preds_final[f'mlp_pred_LOG1P_Z_{i+1}'] = preds

# Listo, df_preds_mlp contiene solo PRODUCT_ID y las 50 columnas de predicción



Prediciendo MLP 1/50...
Prediciendo MLP 2/50...
Prediciendo MLP 3/50...
Prediciendo MLP 4/50...
Prediciendo MLP 5/50...
Prediciendo MLP 6/50...
Prediciendo MLP 7/50...
Prediciendo MLP 8/50...
Prediciendo MLP 9/50...
Prediciendo MLP 10/50...
Prediciendo MLP 11/50...
Prediciendo MLP 12/50...
Prediciendo MLP 13/50...
Prediciendo MLP 14/50...
Prediciendo MLP 15/50...
Prediciendo MLP 16/50...
Prediciendo MLP 17/50...
Prediciendo MLP 18/50...
Prediciendo MLP 19/50...
Prediciendo MLP 20/50...
Prediciendo MLP 21/50...
Prediciendo MLP 22/50...
Prediciendo MLP 23/50...
Prediciendo MLP 24/50...
Prediciendo MLP 25/50...
Prediciendo MLP 26/50...
Prediciendo MLP 27/50...
Prediciendo MLP 28/50...
Prediciendo MLP 29/50...
Prediciendo MLP 30/50...
Prediciendo MLP 31/50...
Prediciendo MLP 32/50...
Prediciendo MLP 33/50...
Prediciendo MLP 34/50...
Prediciendo MLP 35/50...
Prediciendo MLP 36/50...
Prediciendo MLP 37/50...
Prediciendo MLP 38/50...
Prediciendo MLP 39/50...
Prediciendo MLP 40/50...
Predicien

In [20]:
df_preds_final.to_csv("predicciones_mlp_50_modelos.csv", index=False)

In [ ]:
# Imprimir Solo las columnas PRODUCT_ID, mlp_pred_LOG1P_Z_1 a mlp_pred_LOG1P_Z_10  mostrando todas las columnas dando mas ancho 
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.4f}'.format)
print(df_preds_final[['PRODUCT_ID'] + [f'mlp_pred_LOG1P_Z_{i+1}' for i in range(len(configs))]].head())

In [21]:
import gc
import optuna
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor, early_stopping
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- SETUP ---
target_col = 'CLASE_LOG1P_Z'
feature_cols = [col for col in df_train.columns if col != target_col]

X_tr = df_train[feature_cols]
y_tr = df_train[target_col]
X_val = df_val[feature_cols]
y_val = df_val[target_col]

# --- OPTUNA OBJECTIVE ---
def objective_lgbm(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.0005, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 16, 1024),
        "max_depth": trial.suggest_int("max_depth", 4, 64),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
        "random_state": trial.number,   # Distinto para cada trial
        "n_jobs": -1,
        "verbosity": -1,  # Mínimo verbose en LightGBM
    }
    model = LGBMRegressor(**params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        callbacks=[early_stopping(stopping_rounds=30, verbose=False)],
    )
    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    trial.set_user_attr("mae", mean_absolute_error(y_val, preds))
    gc.collect()
    return rmse

# --- OPTIMIZE ---
N_MODELS = 50
N_TRIALS = 1000  

study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=101))
study.optimize(objective_lgbm, n_trials=N_TRIALS, n_jobs=28, show_progress_bar=False)

# --- EXTRACT BEST 50 PARAMS ---
trials_df = study.trials_dataframe()
trials_df["mae"] = [t.user_attrs.get("mae", np.nan) for t in study.trials]

top_lgbm_trials = trials_df.sort_values("value").head(N_MODELS)
final_configs = []
for i, row in top_lgbm_trials.iterrows():
    params = row.filter(like='params_').to_dict()
    params = {k.replace('params_', ''): v for k, v in params.items()}
    for p in ["num_leaves", "max_depth", "n_estimators", "min_child_samples"]:
        params[p] = int(params[p])
    params["random_state"] = int(row["number"])
    params["n_jobs"] = -1
    params["verbosity"] = -1
    final_configs.append(params)

# --- GUARDADO ---
top_lgbm_trials.to_csv("optuna_lgbm_trials.csv", index=False)
import json
with open("lgbm_ensemble_configs.json", "w") as f:
    json.dump(final_configs, f, indent=2)

# --- PRINT RESUMEN SIMPLE ---
print(f"Top 50 RMSE: {top_lgbm_trials['value'].head(50).values}")
print(f"Top 50 MAE: {top_lgbm_trials['mae'].head(50).values}")
print(final_configs)



/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-07-14 21:00:02,343] A new study created in memory with name: no-name-19783afa-3e71-42d4-ac89-52cc307caf94
[I 2025-07-14 21:00:04,740] Trial 27 finished with value: 0.26794116396511897 and parameters: {'learning_rate': 0.29621325470950793, 'num_leaves': 172, 'max_depth': 55, 'n_estimators': 242, 'min_child_samples': 100, 'subsample': 0.8331396622968026, 'colsample_bytree': 0.8467286616297266, 'reg_alpha': 8.661124926901671, 'reg_lambda': 2.1422454083446133e-08, 'min_split_gain': 0.6717672730288906}. Best is trial 27 with value: 0.26794116396511897.
[I 2025-07-14 21:00:06,245] Trial 24 finished with value: 0.2654017373866263 and parameters: {'learning_rate': 0.2989534871331896, 'num_leaves': 511, 'max_depth': 51, 'n_estimators': 792, 'min_

Top 50 RMSE: [0.24977327 0.24990692 0.24995379 0.25126267 0.25173028 0.25192593
 0.25194562 0.2520309  0.25203113 0.2520312  0.2521292  0.25224717
 0.25237975 0.25241549 0.25242355 0.25243109 0.25248236 0.2525209
 0.25259965 0.2527355  0.25274417 0.2528003  0.25285814 0.25291141
 0.25297366 0.25303877 0.25306023 0.25312867 0.25317722 0.25318016
 0.25326933 0.25327496 0.25328069 0.25332354 0.25334156 0.25336186
 0.25338393 0.2534046  0.2534897  0.25351643 0.25354331 0.2535439
 0.25354766 0.25361671 0.25362168 0.25363164 0.25363974 0.25364144
 0.2536421  0.25371085]
Top 50 MAE: [0.16771066 0.16725687 0.16692373 0.1693834  0.16932839 0.16785608
 0.1691616  0.1694729  0.1693808  0.16961925 0.17041808 0.17036967
 0.17043486 0.1702671  0.16996913 0.16940992 0.16845827 0.17037084
 0.17013398 0.17099144 0.17006655 0.16948808 0.17113069 0.17064542
 0.1698298  0.17183973 0.16992524 0.17036683 0.17079193 0.17027244
 0.17081545 0.17145304 0.17066613 0.17095934 0.17126263 0.17135534
 0.17049999 0.1

In [23]:
print(final_configs)

[{'colsample_bytree': 0.5333434673612824, 'learning_rate': 0.06231644339727185, 'max_depth': 58, 'min_child_samples': 58, 'min_split_gain': 0.002062115270766521, 'n_estimators': 844, 'num_leaves': 973, 'reg_alpha': 3.7371206169171867, 'reg_lambda': 7.172199174507102, 'subsample': 0.7971670807732923, 'random_state': 249, 'n_jobs': -1, 'verbosity': -1}, {'colsample_bytree': 0.5228280861398271, 'learning_rate': 0.09415473889227101, 'max_depth': 59, 'min_child_samples': 39, 'min_split_gain': 0.0002401704454612645, 'n_estimators': 867, 'num_leaves': 901, 'reg_alpha': 5.897861800674065, 'reg_lambda': 7.021842519723051, 'subsample': 0.8212381838938069, 'random_state': 588, 'n_jobs': -1, 'verbosity': -1}, {'colsample_bytree': 0.5270720821346941, 'learning_rate': 0.0778624078311394, 'max_depth': 57, 'min_child_samples': 22, 'min_split_gain': 0.0010098246257560799, 'n_estimators': 861, 'num_leaves': 878, 'reg_alpha': 3.313199576803739, 'reg_lambda': 0.7724226921834203, 'subsample': 0.73274595679

In [29]:
X_full = pd.concat([df_train, df_val], ignore_index=True)[feature_cols]
y_full = pd.concat([df_train[target_col], df_val[target_col]], ignore_index=True)

In [30]:
import joblib
from lightgbm import LGBMRegressor

def train_and_save_model(i, params, X_full, y_full):
    print(f"Entrenando modelo {i+1}/50...")
    model = LGBMRegressor(**params)
    model.fit(X_full, y_full)
    joblib.dump(model, f'lgbm_model_{i+1:02d}.pkl')
    return f"Modelo {i+1} terminado"

results = joblib.Parallel(n_jobs=20)(
    joblib.delayed(train_and_save_model)(i, params, X_full, y_full)
    for i, params in enumerate(final_configs[:50])
)

print(results)
print("¡Entrenamiento y guardado de los 50 modelos finalizado!")



Entrenando modelo 1/50...
Entrenando modelo 5/50...
Entrenando modelo 2/50...
Entrenando modelo 3/50...
Entrenando modelo 7/50...
Entrenando modelo 11/50...
Entrenando modelo 13/50...
Entrenando modelo 12/50...
Entrenando modelo 16/50...
Entrenando modelo 9/50...
Entrenando modelo 4/50...
Entrenando modelo 6/50...
Entrenando modelo 8/50...
Entrenando modelo 15/50...
Entrenando modelo 10/50...
Entrenando modelo 14/50...
Entrenando modelo 17/50...
Entrenando modelo 18/50...
Entrenando modelo 19/50...
Entrenando modelo 20/50...
Entrenando modelo 21/50...
Entrenando modelo 22/50...
Entrenando modelo 23/50...
Entrenando modelo 24/50...
Entrenando modelo 25/50...
Entrenando modelo 26/50...
Entrenando modelo 27/50...
Entrenando modelo 28/50...
Entrenando modelo 29/50...
Entrenando modelo 30/50...
Entrenando modelo 31/50...
Entrenando modelo 32/50...
Entrenando modelo 33/50...
Entrenando modelo 34/50...
Entrenando modelo 35/50...
Entrenando modelo 36/50...
Entrenando modelo 37/50...
Entrenando

In [31]:
import joblib

# Cargar los 50 modelos LightGBM entrenados
lgbm_models = []
for i in range(1, 51):
    model = joblib.load(f'lgbm_model_{i:02d}.pkl')
    lgbm_models.append(model)


In [32]:
import numpy as np

# --- Predicción LightGBM (50 modelos) ---
lgbm_preds = []

feature_cols = [col for col in df_pred.columns if col != target_col]

# Asegurarse de que las columnas categóricas tengan el mismo dtype que en el entrenamiento
for col in cat_cols:
    if col in df_pred.columns and col in df_train.columns:
        df_pred[col] = df_pred[col].astype(df_train[col].dtype)

X_pred_lgbm = df_pred[feature_cols]  

for i, model in enumerate(lgbm_models):
    print(f"Prediciendo LightGBM {i+1}/50...")
    preds = model.predict(X_pred_lgbm)
    lgbm_preds.append(preds)

lgbm_preds = np.stack(lgbm_preds).T  # shape (N, 50)

# --- Agregar predicciones LGBM al DataFrame existente con resultados de MLP ---
for i in range(50):
    df_preds_final[f'lgbm_pred_LOG1P_Z_{i+1}'] = lgbm_preds[:, i]



Prediciendo LightGBM 1/50...
Prediciendo LightGBM 2/50...
Prediciendo LightGBM 3/50...
Prediciendo LightGBM 4/50...
Prediciendo LightGBM 5/50...
Prediciendo LightGBM 6/50...
Prediciendo LightGBM 7/50...
Prediciendo LightGBM 8/50...
Prediciendo LightGBM 9/50...
Prediciendo LightGBM 10/50...
Prediciendo LightGBM 11/50...
Prediciendo LightGBM 12/50...
Prediciendo LightGBM 13/50...
Prediciendo LightGBM 14/50...
Prediciendo LightGBM 15/50...
Prediciendo LightGBM 16/50...
Prediciendo LightGBM 17/50...
Prediciendo LightGBM 18/50...
Prediciendo LightGBM 19/50...
Prediciendo LightGBM 20/50...
Prediciendo LightGBM 21/50...
Prediciendo LightGBM 22/50...
Prediciendo LightGBM 23/50...
Prediciendo LightGBM 24/50...
Prediciendo LightGBM 25/50...
Prediciendo LightGBM 26/50...
Prediciendo LightGBM 27/50...
Prediciendo LightGBM 28/50...
Prediciendo LightGBM 29/50...
Prediciendo LightGBM 30/50...
Prediciendo LightGBM 31/50...
Prediciendo LightGBM 32/50...
Prediciendo LightGBM 33/50...
Prediciendo LightGB

/tmp/ipykernel_656/4155869407.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_preds_final[f'lgbm_pred_LOG1P_Z_{i+1}'] = lgbm_preds[:, i]


In [43]:
print(df_train.columns.to_list())

['PERIODO', 'MES_SIN', 'MES_COS', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'PRODUCT_ID', 'MES_PROBLEMATICO', 'IS_FEBRERO', 'ESTOY_PREDICIENDO_FEBRERO', 'CAIDA_ABRUPTA', 'CLASE_LOG1P_Z', 'PRODUCT_RANK_BIN', 'ORDINAL_Z', 'ANIO_Z', 'MES_Z', 'TRIMESTRE_Z', 'PRODUCT_ID_Z', 'TN_Z', 'PROM_ULT_3_FEBREROS_Z', 'DIF_TN_VS_FEBREROS_ULT_3_Z', 'TN_MAX_HISTORICO_Z', 'TN_DIST_A_MAX_HIST_Z', 'TN_RATIO_VS_MAX_HIST_Z', 'TN_MEAN_03_Z', 'PENDIENTE_TENDENCIA_3_Z', 'TN_EWMA_03_Z', 'TN_MEDIAN_03_Z', 'TN_MIN_03_Z', 'TN_MAX_03_Z', 'TN_STD_03_Z', 'TN_SKEW_03_Z', 'TN_KURT_03_Z', 'TN_GROWTH_03_Z', 'TN_IQR_03_Z', 'TN_SUM_03_Z', 'TN_COUNT_POS_03_Z', 'TN_PCT_ZERO_03_Z', 'TN_LAST_03_Z', 'TN_LAST_DIFF_03_Z', 'TN_COEF_VAR_3_Z', 'TN_MAXMIN_RATIO_3_Z', 'TN_RANGO_3_Z', 'TN_RANGO_REL_3_Z', 'TN_LAST_VS_MEDIAN_3_Z', 'TN_CHANGE_PREV_WINDOW_3_Z', 'TN_ZEROS_END_3_Z', 'TN_LAST_PCT_SUM_3_Z', 'TN_PCT90_3_Z', 'TN_PCT10_3_Z', 'TN_PCT_WIDTH_3_Z', 'TN_MINUS_MEAN_03_Z', 'TN_MINUS_MEDIAN_03_Z', 'TN_MINUS_EWMA_03_Z', 'TN_O

In [33]:
# Imprimir Solo las columnas PRODUCT_ID, mlp_pred_LOG1P_Z_1 a mlp_pred_LOG1P_Z_10  mostrando todas las columnas dando mas ancho 
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.4f}'.format)
print(df_preds_final.head(50))

    PRODUCT_ID  mlp_pred_LOG1P_Z_1  mlp_pred_LOG1P_Z_2  mlp_pred_LOG1P_Z_3  mlp_pred_LOG1P_Z_4  mlp_pred_LOG1P_Z_5  mlp_pred_LOG1P_Z_6  mlp_pred_LOG1P_Z_7  mlp_pred_LOG1P_Z_8  mlp_pred_LOG1P_Z_9  mlp_pred_LOG1P_Z_10  mlp_pred_LOG1P_Z_11  mlp_pred_LOG1P_Z_12  mlp_pred_LOG1P_Z_13  mlp_pred_LOG1P_Z_14  mlp_pred_LOG1P_Z_15  mlp_pred_LOG1P_Z_16  mlp_pred_LOG1P_Z_17  mlp_pred_LOG1P_Z_18  mlp_pred_LOG1P_Z_19  mlp_pred_LOG1P_Z_20  mlp_pred_LOG1P_Z_21  mlp_pred_LOG1P_Z_22  mlp_pred_LOG1P_Z_23  mlp_pred_LOG1P_Z_24  mlp_pred_LOG1P_Z_25  mlp_pred_LOG1P_Z_26  mlp_pred_LOG1P_Z_27  mlp_pred_LOG1P_Z_28  mlp_pred_LOG1P_Z_29  mlp_pred_LOG1P_Z_30  mlp_pred_LOG1P_Z_31  mlp_pred_LOG1P_Z_32  mlp_pred_LOG1P_Z_33  mlp_pred_LOG1P_Z_34  mlp_pred_LOG1P_Z_35  mlp_pred_LOG1P_Z_36  mlp_pred_LOG1P_Z_37  mlp_pred_LOG1P_Z_38  mlp_pred_LOG1P_Z_39  mlp_pred_LOG1P_Z_40  mlp_pred_LOG1P_Z_41  mlp_pred_LOG1P_Z_42  mlp_pred_LOG1P_Z_43  mlp_pred_LOG1P_Z_44  mlp_pred_LOG1P_Z_45  mlp_pred_LOG1P_Z_46  mlp_pred_LOG1P_Z_47  mlp_pr

In [34]:
# Cargar desde disco
scaler_y = joblib.load('scaler_y_CLASE_LOG1P.joblib')
#y_pred_log1p = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()

In [35]:
def inverse_log_transform_signed(x):
    return np.sign(x) * np.expm1(np.abs(x))

def inv_transform_with_scaler(arr, scaler_y):
    # arr debe tener forma (N,)
    arr = arr.reshape(-1, 1)  # scaler espera 2D
    # Paso 1: inversa del escalado
    log1p_vals = scaler_y.inverse_transform(arr).flatten()
    # Paso 2: inversa de log1p con signo
    orig_vals = inverse_log_transform_signed(log1p_vals)
    return orig_vals

# Crear columnas nuevas con sufijo "_ORIG"
for col in df_preds_final.columns:
    if col not in ['CUSTOMER_ID', 'PRODUCT_ID']:
        arr = df_preds_final[col].values
        df_preds_final[col.replace('_LOG1P_Z_', '_ORIG_')] = inv_transform_with_scaler(arr, scaler_y)

/tmp/ipykernel_656/3128550729.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_preds_final[col.replace('_LOG1P_Z_', '_ORIG_')] = inv_transform_with_scaler(arr, scaler_y)
/tmp/ipykernel_656/3128550729.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_preds_final[col.replace('_LOG1P_Z_', '_ORIG_')] = inv_transform_with_scaler(arr, scaler_y)
/tmp/ipykernel_656/3128550729.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performanc

In [36]:
# Eliminar de df_preds_final las columnas con "_LOG1P_Z_"
cols_to_remove = [col for col in df_preds_final.columns if '_LOG1P_Z_' in col]
df_preds_final.drop(columns=cols_to_remove, inplace=True)

In [38]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.4f}'.format)
display(df_preds_final.head(50))

,PRODUCT_ID,mlp_pred_ORIG_1,mlp_pred_ORIG_2,mlp_pred_ORIG_3,mlp_pred_ORIG_4,mlp_pred_ORIG_5,mlp_pred_ORIG_6,mlp_pred_ORIG_7,mlp_pred_ORIG_8,mlp_pred_ORIG_9,mlp_pred_ORIG_10,mlp_pred_ORIG_11,mlp_pred_ORIG_12,mlp_pred_ORIG_13,mlp_pred_ORIG_14,mlp_pred_ORIG_15,mlp_pred_ORIG_16,mlp_pred_ORIG_17,mlp_pred_ORIG_18,mlp_pred_ORIG_19,mlp_pred_ORIG_20,mlp_pred_ORIG_21,mlp_pred_ORIG_22,mlp_pred_ORIG_23,mlp_pred_ORIG_24,mlp_pred_ORIG_25,mlp_pred_ORIG_26,mlp_pred_ORIG_27,mlp_pred_ORIG_28,mlp_pred_ORIG_29,mlp_pred_ORIG_30,mlp_pred_ORIG_31,mlp_pred_ORIG_32,mlp_pred_ORIG_33,mlp_pred_ORIG_34,mlp_pred_ORIG_35,mlp_pred_ORIG_36,mlp_pred_ORIG_37,mlp_pred_ORIG_38,mlp_pred_ORIG_39,mlp_pred_ORIG_40,mlp_pred_ORIG_41,mlp_pred_ORIG_42,mlp_pred_ORIG_43,mlp_pred_ORIG_44,mlp_pred_ORIG_45,mlp_pred_ORIG_46,mlp_pred_ORIG_47,mlp_pred_ORIG_48,mlp_pred_ORIG_49,mlp_pred_ORIG_50,lgbm_pred_ORIG_1,lgbm_pred_ORIG_2,lgbm_pred_ORIG_3,lgbm_pred_ORIG_4,lgbm_pred_ORIG_5,lgbm_pred_ORIG_6,lgbm_pred_ORIG_7,lgbm_pred_ORIG_8,lgbm_pred_ORIG_9,lgbm_pred_ORIG_10,lgbm_pred_ORIG_11,lgbm_pred_ORIG_12,lgbm_pred_ORIG_13,lgbm_pred_ORIG_14,lgbm_pred_ORIG_15,lgbm_pred_ORIG_16,lgbm_pred_ORIG_17,lgbm_pred_ORIG_18,lgbm_pred_ORIG_19,lgbm_pred_ORIG_20,lgbm_pred_ORIG_21,lgbm_pred_ORIG_22,lgbm_pred_ORIG_23,lgbm_pred_ORIG_24,lgbm_pred_ORIG_25,lgbm_pred_ORIG_26,lgbm_pred_ORIG_27,lgbm_pred_ORIG_28,lgbm_pred_ORIG_29,lgbm_pred_ORIG_30,lgbm_pred_ORIG_31,lgbm_pred_ORIG_32,lgbm_pred_ORIG_33,lgbm_pred_ORIG_34,lgbm_pred_ORIG_35,lgbm_pred_ORIG_36,lgbm_pred_ORIG_37,lgbm_pred_ORIG_38,lgbm_pred_ORIG_39,lgbm_pred_ORIG_40,lgbm_pred_ORIG_41,lgbm_pred_ORIG_42,lgbm_pred_ORIG_43,lgbm_pred_ORIG_44,lgbm_pred_ORIG_45,lgbm_pred_ORIG_46,lgbm_pred_ORIG_47,lgbm_pred_ORIG_48,lgbm_pred_ORIG_49,lgbm_pred_ORIG_50
0,20001,2499.2090,5263.2959,804.5912,727.6192,1559.3298,2911.9277,7580.3813,932.1174,2122.7937,2514.7844,4620.2842,2722.4951,4365.8901,1492.4357,4894.0991,1080.9789,728.9855,3817.6021,2046.8887,1205.3357,1795.4785,917.8847,2162.9702,2008.6067,3880.3560,7916.3921,2108.4348,14830.9072,3071.4585,3162.0374,4540.4404,4199.3940,627.3073,3711.7502,10808.7129,4010.7842,2787.4878,1962.2367,3214.1221,3062.6428,3114.7310,1005.9449,4292.7256,2789.5337,227.9098,1334.6522,1259.5654,1528.6150,2745.5435,1266.5941,935.1566,924.6894,902.1176,848.3029,847.1355,1157.0333,886.3787,840.6570,907.4833,997.5659,939.1530,1088.7173,1124.7459,960.0096,952.6442,912.2195,1110.8863,869.8642,1243.7613,897.8728,896.6645,954.6576,885.1398,913.2840,889.9761,890.7729,996.9416,879.9595,1193.1274,1006.1139,995.2010,881.5662,982.9409,833.5339,913.6344,822.5711,847.4863,1046.9595,949.3221,906.5048,845.7527,923.6794,1223.6973,906.0816,1274.6605,1021.0181,1118.0922,866.9102,870.2219,1073.2910
1,20002,473.9576,2823.5747,600.2195,562.8087,814.6390,766.4780,1546.3629,697.9310,1351.1823,861.1847,1697.4403,871.4789,833.1289,986.6110,2509.4326,485.7554,377.7762,1358.7043,708.0589,702.3293,704.1918,505.4628,1177.1813,1257.8733,1018.1789,2839.2078,907.6165,1758.9507,1266.1028,1861.8225,1645.7792,1898.1858,374.1505,1872.4506,2554.7678,2109.0835,1151.9299,1068.7759,1398.7549,735.1407,438.5002,396.7874,1140.4252,1037.7700,721.7929,469.1016,733.5745,606.8829,1220.3689,514.1923,981.6201,939.5924,925.6344,905.8875,857.1993,1066.8393,858.0403,862.9338,933.1881,996.1841,986.4728,987.0411,990.3274,976.5534,974.2095,905.7628,1023.4710,897.3699,981.4370,918.8154,909.6917,950.0992,853.5017,940.0597,884.6668,912.2222,967.2535,929.4169,1016.0062,1004.1714,931.8859,900.4557,908.7761,864.7181,918.5266,830.9508,943.2141,990.5508,966.8951,951.4487,890.2561,935.1005,1031.1569,977.4998,1042.5057,1091.4635,1072.1501,882.6033,903.1474,1013.8543
2,20003,883.6436,1229.2775,341.9880,209.2350,570.0003,709.4800,1465.1033,475.2508,871.2306,771.3541,1210.7092,844.2320,1276.0896,455.5036,1272.2114,446.6269,334.5750,1082.2994,741.9630,457.5691,783.3557,551.6803,694.3302,812.8923,917.5224,1999.0488,685.9417,1526.1064,879.1579,939.0835,786.8970,1354.7971,190.1784,1385.3892,1266.3289,974.0811,875.19

In [39]:
# Columnas de predicción

mlp_cols = [c for c in df_preds_final.columns if c.startswith('mlp_pred_ORIG_')]
lgbm_cols = [c for c in df_preds_final.columns if c.startswith('lgbm_pred_ORIG_')]

df_preds_final['mlp_median'] = df_preds_final[mlp_cols].median(axis=1)
df_preds_final['lgbm_median'] = df_preds_final[lgbm_cols].median(axis=1)
df_preds_final['ensemble_median'] = df_preds_final[mlp_cols + lgbm_cols].median(axis=1)

# Medias (nuevo)
df_preds_final['mlp_mean'] = df_preds_final[mlp_cols].mean(axis=1)
df_preds_final['lgbm_mean'] = df_preds_final[lgbm_cols].mean(axis=1)
df_preds_final['ensemble_mean'] = df_preds_final[mlp_cols + lgbm_cols].mean(axis=1)

# Desvío estándar
df_preds_final['mlp_std'] = df_preds_final[mlp_cols].std(axis=1)
df_preds_final['lgbm_std'] = df_preds_final[lgbm_cols].std(axis=1)
df_preds_final['ensemble_std'] = df_preds_final[mlp_cols + lgbm_cols].std(axis=1)

/tmp/ipykernel_656/1127774767.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_preds_final['mlp_median'] = df_preds_final[mlp_cols].median(axis=1)
/tmp/ipykernel_656/1127774767.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_preds_final['lgbm_median'] = df_preds_final[lgbm_cols].median(axis=1)
/tmp/ipykernel_656/1127774767.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using 

In [40]:
print(df_preds_final)

     PRODUCT_ID  mlp_pred_ORIG_1  mlp_pred_ORIG_2  mlp_pred_ORIG_3  mlp_pred_ORIG_4  mlp_pred_ORIG_5  mlp_pred_ORIG_6  mlp_pred_ORIG_7  mlp_pred_ORIG_8  mlp_pred_ORIG_9  mlp_pred_ORIG_10  mlp_pred_ORIG_11  mlp_pred_ORIG_12  mlp_pred_ORIG_13  mlp_pred_ORIG_14  mlp_pred_ORIG_15  mlp_pred_ORIG_16  mlp_pred_ORIG_17  mlp_pred_ORIG_18  mlp_pred_ORIG_19  mlp_pred_ORIG_20  mlp_pred_ORIG_21  mlp_pred_ORIG_22  mlp_pred_ORIG_23  mlp_pred_ORIG_24  mlp_pred_ORIG_25  mlp_pred_ORIG_26  mlp_pred_ORIG_27  mlp_pred_ORIG_28  mlp_pred_ORIG_29  mlp_pred_ORIG_30  mlp_pred_ORIG_31  mlp_pred_ORIG_32  mlp_pred_ORIG_33  mlp_pred_ORIG_34  mlp_pred_ORIG_35  mlp_pred_ORIG_36  mlp_pred_ORIG_37  mlp_pred_ORIG_38  mlp_pred_ORIG_39  mlp_pred_ORIG_40  mlp_pred_ORIG_41  mlp_pred_ORIG_42  mlp_pred_ORIG_43  mlp_pred_ORIG_44  mlp_pred_ORIG_45  mlp_pred_ORIG_46  mlp_pred_ORIG_47  mlp_pred_ORIG_48  mlp_pred_ORIG_49  mlp_pred_ORIG_50  lgbm_pred_ORIG_1  lgbm_pred_ORIG_2  lgbm_pred_ORIG_3  lgbm_pred_ORIG_4  lgbm_pred_ORIG_5  lg

In [41]:
df_preds_final.to_csv('predicciones_finales.csv', index=False)